In [1]:
import json

- 算法流程（ppo，grpo）
- 懂 verl 的工作流程
    - verl 的数据结构设计；
        - DataProto：batch、non_batch_tensor

```sh
set -x

export VLLM_ATTENTION_BACKEND=XFORMERS

NUM_GPUS=$(nvidia-smi --query-gpu=name --format=csv,noheader | wc -l)

ray stop
sleep 5

ray start --head --node-ip-address 0.0.0.0 --num-gpus $NUM_GPUS --ray-debugger-external --port 6380

python3 -m verl.trainer.main_ppo \
    algorithm.adv_estimator=grpo \
    data.train_files=$HOME/data/gsm8k/train.parquet \
    data.val_files=$HOME/data/gsm8k/test.parquet \
    data.train_batch_size=8 \
    data.max_prompt_length=256 \
    data.max_response_length=512 \
    actor_rollout_ref.model.path=Qwen/Qwen2.5-0.5B-Instruct \
    actor_rollout_ref.actor.optim.lr=1e-6 \
    actor_rollout_ref.model.use_remove_padding=True \
    actor_rollout_ref.actor.ppo_mini_batch_size=8 \
    actor_rollout_ref.actor.ppo_micro_batch_size_per_gpu=4 \
    actor_rollout_ref.actor.use_kl_loss=True \
    actor_rollout_ref.actor.kl_loss_coef=0.001 \
    actor_rollout_ref.actor.kl_loss_type=low_var_kl \
    actor_rollout_ref.model.enable_gradient_checkpointing=True \
    actor_rollout_ref.actor.fsdp_config.param_offload=False \
    actor_rollout_ref.actor.fsdp_config.optimizer_offload=False \
    actor_rollout_ref.rollout.log_prob_micro_batch_size_per_gpu=160 \
    actor_rollout_ref.rollout.tensor_model_parallel_size=1 \
    actor_rollout_ref.rollout.name=vllm \
    actor_rollout_ref.rollout.gpu_memory_utilization=0.6 \
    actor_rollout_ref.rollout.n=2 \
    actor_rollout_ref.ref.log_prob_micro_batch_size_per_gpu=160 \
    actor_rollout_ref.ref.fsdp_config.param_offload=True \
    algorithm.kl_ctrl.kl_coef=0.001 \
    trainer.critic_warmup=0 \
    trainer.logger=['console','wandb'] \
    trainer.project_name='verl_grpo_example_gsm8k' \
    trainer.experiment_name='qwen_0.5b_function_rm' \
    trainer.n_gpus_per_node=$NUM_GPUS \
    trainer.nnodes=1 \
    trainer.save_freq=-1 \
    trainer.test_freq=5 \
    trainer.total_epochs=15 $@
```

- main_ppo.py

```python
trainer = RayPPOTrainer(config=config,
                        tokenizer=tokenizer,
                        role_worker_mapping=role_worker_mapping,
                        resource_pool_manager=resource_pool_manager,
                        ray_worker_group_cls=ray_worker_group_cls,
                        reward_fn=reward_fn,
                        val_reward_fn=val_reward_fn)
trainer.init_workers()
trainer.fit()
```

## data

- tokenizer => apply_chat_template
    - vllm parameters
- data_source
    - 一份 val，可以 merge 多个 data_source 来源的数据
 
```json
{
    'data_source': 'openai/gsm8k', 
    'ability': 'math', 
    'reward_model': {'ground_truth': '18', 'style': 'rule'}, 
    'extra_info': {'answer': 'Janet sells 16 - 3 -...t.\n#### 18', 'index': 0, 'question': 'Janet’s ducks lay 16... market?", 'split': 'test'}, 
    'index': 0
}
```

### DataProto

```
batch: TensorDict = None
non_tensor_batch: Dict = field(default_factory=dict)
meta_info: Dict = field(default_factory=dict)
```

## `_validate`

- `Validation datasets are sent to inference engines as a whole batch, which will schedule the memory themselves.`
    - vllm inference engine
- batch
    - left padding, 151643, `<|endoftext|>`
        - 198, '\n'

### ppo training

### metrics

- actor
    - ppo_kl (inside PPO (RLHF)):
        ```python
        negative_approx_kl = log_prob - old_log_prob
        ratio = torch.exp(negative_approx_kl)
        ppo_kl = verl_F.masked_mean(-negative_approx_kl, eos_mask)
        ```
    - kl_loss: $kl(\pi_\theta\|\pi_{ref})$